## Loading and Prepping the Data

In [83]:
import pandas as pd

In [84]:
data=pd.read_csv('data/phonemeData.txt', sep='\t', header=None)


In [85]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,727,728,729,730,731,732,733,734,735,736
0,1,Hadza,hts,hts-TZA,Khoisan,800,Africa,-3.47684,34.79905,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Sandawe,sad,sad-TZA,Khoisan,40000,Africa,-5.28543,35.46569,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Qxu^,knw,knw-NAM,Khoisan,1760,Africa,-19.47014,18.35209,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Qxu^,ktz,ktz-NAM,Khoisan,28600,Africa,-19.76648,20.09317,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Qxu^,ktz,ktz-NAM,Khoisan,28600,Africa,-19.76648,20.09317,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
newColumns = data.columns.tolist()


In [87]:
newColumns[0] = "Language Number"
newColumns[1] = "Name"
newColumns[2] = "ISO Code"
newColumns[3] = "ISO A3 Code"
newColumns[4] = "Language Classification"
newColumns[5] = "Speaking Pop Size"
newColumns[6] = "Geographic Region"
newColumns[7] = "Latitude"
newColumns[8] = "Longitude"
for i in range(728):
    startPhonemeIndex = 9
    newColumns[startPhonemeIndex + i] = "Phoneme " + str(i)

#newColumns

In [88]:
data.columns = newColumns
data.head()

,Language Number,Name,ISO Code,ISO A3 Code,Language Classification,Speaking Pop Size,Geographic Region,Latitude,Longitude,Phoneme 0,...,Phoneme 718,Phoneme 719,Phoneme 720,Phoneme 721,Phoneme 722,Phoneme 723,Phoneme 724,Phoneme 725,Phoneme 726,Phoneme 727
0,1,Hadza,hts,hts-TZA,Khoisan,800,Africa,-3.47684,34.79905,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Sandawe,sad,sad-TZA,Khoisan,40000,Africa,-5.28543,35.46569,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Qxu^,knw,knw-NAM,Khoisan,1760,Africa,-19.47014,18.35209,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Qxu^,ktz,ktz-NAM,Khoisan,28600,Africa,-19.76648,20.09317,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Qxu^,ktz,ktz-NAM,Khoisan,28600,Africa,-19.76648,20.09317,0,...,0,0,0,0,0,0,0,0,0,0


Notes: https://www.ethnologue.com
is where you can paste in the ISO code to see that it's a real language, the "Name" column that is provided might not be super accurate

## Similarity Metrics

### Hamming Distance

### Jaccard Similarity

## Visualization 

## Clustering Analysis